In [63]:
import pandas as pd
import numpy as np
import ast
import json

In [48]:
# Load in all 2 million rows
full_df = pd.read_csv('full_dataset.csv')

In [49]:
# Get rid of rows with null values and get the first 50,000 rows
clean_df = full_df.dropna().head(50000)
# Drop unneccessary columns
clean_df = clean_df.drop(columns=["Unnamed: 0", "source"])

In [50]:
# Convert lists and other nested data to their correct data type from strings
def try_literal_eval(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except Exception:
            return val
    return val

clean_df = clean_df.applymap(try_literal_eval)

/tmp/ipykernel_3273517/2749866470.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clean_df = clean_df.applymap(try_literal_eval)


In [51]:
clean_df["ingredients"] = clean_df["ingredients"].apply(lambda x: " ".join(x))
clean_df["directions"] = clean_df["directions"].apply(lambda x: " ".join(x))
clean_df["NER"] = clean_df["NER"].apply(lambda x: ", ".join(x))

In [52]:
# Join all the relevant recipe columns into a single column
clean_df["full_doc"] = clean_df.apply(lambda x: "; ".join([x["title"],x["NER"], x["ingredients"], x["directions"]]), axis = 1)

In [59]:
clean_df.insert(0, "docid", range(len(clean_df)))

In [61]:
output_file = "project_data.json"

In [64]:
# save data as a json
with open(output_file, "w", encoding="utf-8") as f:
    for _, row in clean_df.iterrows():
        obj = row.to_dict()
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")